Aula 1 - Do código intermediário ao executável

In [1]:
# Simulação de código intermediário (TAC) e tradução para código de máquina

# Simulação do código intermediário TAC
tac_code = [
    "t1 = a + b",  # t1 = a + b
    "t2 = t1 * c",  # t2 = t1 * c
    "result = t2 + d"  # result = t2 + d
]

# Função para traduzir TAC para instruções de máquina (usando registradores)
def translate_to_machine_code(tac, variables):
    # Inicializa os registradores
    registers = {f"R{i}": 0 for i in range(4)}  # Registradores R0, R1, R2, R3
    machine_code = []  # Lista para armazenar o código de máquina gerado

    # Mapear variáveis para registradores
    memory = variables.copy()  # Inicializa a memória com as variáveis fornecidas

    # Traduzir as instruções de TAC para código de máquina
    for line in tac:
        if '=' in line:
            # Se a instrução for uma atribuição
            var, expr = line.split('=')
            var = var.strip()
            expr = expr.strip()

            if '+' in expr:  # Soma
                left, right = expr.split('+')
                left = memory.get(left.strip(), 0)
                right = memory.get(right.strip(), 0)
                registers["R1"] = left + right
                machine_code.append(f"ADD R1, {left}, {right}  # {var} = {left} + {right}")
                memory[var] = registers["R1"]

            elif '*' in expr:  # Multiplicação
                left, right = expr.split('*')
                left = memory.get(left.strip(), 0)
                right = memory.get(right.strip(), 0)
                registers["R2"] = left * right
                machine_code.append(f"MUL R2, {left}, {right}  # {var} = {left} * {right}")
                memory[var] = registers["R2"]

            elif expr.isdigit():  # Caso seja um número
                registers["R0"] = int(expr)
                machine_code.append(f"MOV R0, {expr}  # {var} = {expr}")
                memory[var] = registers["R0"]

    return machine_code, memory


# Definindo valores iniciais das variáveis
variables = {
    "a": 5,
    "b": 3,
    "c": 2,
    "d": 7
}

# Traduzindo o código intermediário (TAC) para código de máquina
machine_code, final_memory = translate_to_machine_code(tac_code, variables)

# Exibindo o código de máquina gerado
print("Código de Máquina Gerado:")
for line in machine_code:
    print(line)

print("\nEstado Final dos Registradores:")
for reg, value in final_memory.items():
    print(f"{reg}: {value}")


Código de Máquina Gerado:
ADD R1, 5, 3  # t1 = 5 + 3
MUL R2, 8, 2  # t2 = 8 * 2
ADD R1, 16, 7  # result = 16 + 7

Estado Final dos Registradores:
a: 5
b: 3
c: 2
d: 7
t1: 8
t2: 16
result: 23


Aula 3 - Simulando um código assembly (Colab)

In [2]:
# Simulador simples para código Assembly
class SimpleVM:
    def __init__(self):
        self.regs = {f"R{i}": 0 for i in range(4)}  # Registradores R0, R1, R2, R3

    def MOV(self, reg, value):
        self.regs[reg] = value

    def ADD(self, reg, reg1, reg2):
        self.regs[reg] = self.regs[reg1] + self.regs[reg2]

    def show(self):
        return self.regs

# Executando código Assembly simples
vm = SimpleVM()
vm.MOV("R1", 5)  # R1 = 5
vm.MOV("R2", 3)  # R2 = 3
vm.ADD("R3", "R1", "R2")  # R3 = R1 + R2
print(vm.show())  # Exibindo os valores dos registradores


{'R0': 0, 'R1': 5, 'R2': 3, 'R3': 8}


Aula 4 - Mapeando variáveis para registradores

In [3]:
# Função para simular o mapeamento de variáveis para registradores
def map_variables_to_registers(variables, num_registers):
    # Registradores disponíveis (R0, R1, ..., Rn-1)
    registers = {f"R{i}": None for i in range(num_registers)}
    memory = {}  # Armazena variáveis que não podem ser mapeadas para registradores
    register_map = {}  # Mapeamento das variáveis para registradores

    # Tentar mapear cada variável para um registrador
    for var, value in variables.items():
        # Se houver registradores disponíveis
        assigned = False
        for reg in registers:
            if registers[reg] is None:  # Se o registrador estiver livre
                registers[reg] = value
                register_map[var] = reg
                assigned = True
                break

        if not assigned:  # Se não for possível, move a variável para a memória (spill)
            memory[var] = value

    return register_map, registers, memory

# Exemplo de variáveis e número de registradores
variables = {
    "a": 5,
    "b": 3,
    "c": 7,
    "d": 9,
    "e": 11,
}

num_registers = 3  # Número de registradores disponíveis

# Mapeando as variáveis para os registradores
register_map, registers, memory = map_variables_to_registers(variables, num_registers)

# Exibindo os resultados
print("Mapeamento de Variáveis para Registradores:")
for var, reg in register_map.items():
    print(f"{var} mapeado para {reg}")

print("\nEstado dos Registradores:")
for reg, value in registers.items():
    print(f"{reg}: {value}")

print("\nVariáveis que foram movidas para a Memória (spill):")
for var, value in memory.items():
    print(f"{var}: {value}")

Mapeamento de Variáveis para Registradores:
a mapeado para R0
b mapeado para R1
c mapeado para R2

Estado dos Registradores:
R0: 5
R1: 3
R2: 7

Variáveis que foram movidas para a Memória (spill):
d: 9
e: 11
